# SQL Server Database

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy import text
from dotenv import load_dotenv
from functools import partial
import os
import pandas as pd

In [ ]:
def create_connection_url(server, database, username, password, port, driver):
    connection_url = URL.create(
        "mssql+pyodbc",
        username=username,
        password=password,
        host=server,
        port=port,
        database=database,
        query={
            "driver": driver,
            # "TrustServerCertificate": "yes",
            # "authentication": "ActiveDirectoryIntegrated",
        }
    )
    return connection_url

In [ ]:
def test_connection(connection_string):
    try:
        # Replace the connection string with your own
        connection_string = connection_string
        engine = create_engine(connection_string, echo=True)
        with engine.connect() as conn:
            result = conn.execute(text("SELECT DB_NAME()")).fetchone()
        assert result[0] is not None
        print(f'Connection to {result[0]} was successful.')
    except Exception as e:
        # If the connection fails, the test will fail with an exception
        assert False, f'Connection test failed: {e}'

## Create Engine

In [ ]:
server = os.environ.get('server')
database = 'NBA'
username = os.getenv('DB_username')
password = os.getenv('DB_password')
port = os.getenv('port')
driver = 'ODBC Driver 17 for SQL Server'

CreateURL = partial(create_connection_url, server=server,
                    username=username, password=password, port=port, driver=driver,)

nba_connection = CreateURL(
    database=database)

test_connection(nba_connection)
engine = create_engine(url=nba_connection, echo=True)


## Create Table

### Columns to Create

In [ ]:
player_shotLocations = pd.read_csv(r"D:\lianz\Desktop\Python\personal_projects\nba\data\player_shotLocations.csv")

### Execute SQL to Create Table

In [ ]:
create_table = text("""
CREATE TABLE player_shotLocations(
player_id INT PRIMARY KEY,
y INT
)""")

with engine.begin() as conn:
    conn.execute(create_table)

## Insert Table

### Execute SQL Statement to INSERT

In [ ]:
insert_table = text("""
INSERT INTO player_shotLocations (player_id, y)
VALUES (:player_id, :y)
""")

with engine.begin() as conn:
    conn.execute(insert_table, [{"player_id": 1, "y": 1}, {"player_id": 2, "y": 4}],)

# Drop Table

In [ ]:
drop_table = text("""
DROP TABLE player_shotLocations
""")

with engine.begin() as conn:
    conn.execute(drop_table)